### Implementação de Filtragem Colaborativa com Bibliotecas

Filtragem colaborativa é um processo oriundo de sistemas de recomendação, onde o objetivo é filtrar uma lista de itens para sugerir a um usuário o item mais "relevante" dessa lista.

Itens são avaliados como "relevante" a partir de uma análise que busca por similaridades em um histórico de avaliações.

Aqui, implementamos Filtragem Colaborativa a partir da biblioteca `Surprise`, do Python.

#### 1. Importação de Bibliotecas

In [1]:
from surprise import Dataset, Reader, KNNBasic, dump
from surprise.model_selection import cross_validate

from collections import defaultdict

import numpy as np
import pandas as pd

#### 2. Histórico de Avaliações

Usaremos dados de um histórico de avaliações para os processos subsequentes.

Esses dados serão armazenados em uma tabela (`pandas DataFrame`), onde cada linha é uma avaliação, e as colunas são `"usuario"`, `"item"`, e `"avaliacao"`.

In [2]:
avaliacoes = pd.DataFrame({
    "usuario": [0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 4],
    "item": [0, 1, 3, 4, 0, 3, 3, 0, 3, 5, 2, 3, 5],
    "avaliacao": [5, 3, 4, 4, 1, 3, 1, 4, 5, 2, 5, 4, 1],
})

In [3]:
print(avaliacoes)

    usuario  item  avaliacao
0         0     0          5
1         0     1          3
2         0     3          4
3         0     4          4
4         1     0          1
5         1     3          3
6         2     3          1
7         3     0          4
8         3     3          5
9         3     5          2
10        4     2          5
11        4     3          4
12        4     5          1


Note que os dados acima são idênticos aos utilizados durante a implementação básica de filtragem colaborativa.

O usuário 5, presente na implementação básica de filtragem colaborativa, não aparece aqui. Isso se deve ao fato de que ele não tinha avaliado nenhum item. Essa é uma peculiaridade de sistemas de recomendação colaborativos: usuários que não contribuem para o histórico de avaliações são excluídos do sistema.

#### 3. Criando o objeto da Estrutura de Dados de Avaliações do Surprise

Agora, vamos criar a estrutura de dados de avaliações principal utilizada pelo Surprise. 

Primeiro, precisamos criar um objeto `Reader` (*Leitor*, do Inglês) para ler a nossa `pandas DataFrame`. Passamos `rating_scale=(1, 5)` como argumento para informar o Surprise que nossas avaliações variam de `1` a `5`.

In [4]:
leitor = Reader(rating_scale=(1, 5))

Já conseguimos criar a estrutura de dados `Dataset` do Surprise a partir da linha abaixo:

In [5]:
dados = Dataset.load_from_df(avaliacoes, leitor)

In [6]:
print(dados)

#### 4. Criando o Conjunto de Dados para Treinamento

Até agora, temos um objeto que armazena as nossas avaliações em um formato que o Surprise entende. Precisamos usar esses dados para treinar um modelo do Surprise. Uma vez treinado, o modelo tem a capacidade de avaliar a utilidade de cada item para cada usuário, e portanto gerar recomendações.

Podemos escolher quantos dados da totalidade de dados armazenados em nosso objeto serão utilizados durante esse treinamento. Como são poucos, podemos escolher *todos* os dados disponíveis sem custos computacionais caros.

Para criar um Conjunto de Dados para Treinamento que dispõe de todos os dados de avaliações disponíveis, podemos utilizar a linha abaixo:

In [7]:
conjunto_dados_treinamento = dados.build_full_trainset()

#### 5. Criando e Treinando o Modelo

Para treinar nosso modelo (e, eventualmente, obter a capacidade de avaliar a utilidade de cada item para cada usuário), primeiro precisamos escolher o algoritmo por trás dele.

Em filtragem colaborativa, esse algoritmo é chamado de `KNN`: *k Nearest-Neighbors*, que significa "k Vizinhos Mais Próximos", ou "Mais Similares".

A intuição por trás desse algoritmo é a mesma que da implementação básica discutida anteriormente: estamos procurando por similaridades entre usuários/itens, e dispondo dessas informações para calcular utilidades.

Podemos especificar o método de calculo da similaridade entre usuários e se o algoritmo será baseado em itens ou usuários a partir de um dicionário. Aqui, estamos usando filtragem colaborativa baseada em usuários, e comparando usuários com de similaridade de coseno:

In [8]:
configuracoes = {
    "name": "cosine",
    "user_based": True,
}

Passamos essas configurações para a classe `KNNBasic` quando quisermos criar um objeto de nosso algoritmo na forma do argumento `sim_options=configuracoes`:

In [9]:
algoritmo = KNNBasic(sim_options=configuracoes)

Para treinar nosso algoritmo em nosso conjunto de dados para treinamento (que, novamente, neste caso é o conjunto de *todos* os dados de avaliações disponíveis), basta utilizar o método `fit` do objeto, passando o `conjunto_dados_treinamento` como argumento:

In [10]:
algoritmo.fit(conjunto_dados_treinamento)

Computing the cosine similarity matrix...
Done computing similarity matrix.


#### 6. Avaliando a Utilidade

Para obter, por exemplo, uma estimativa da avaliação que o usuário 1 daria ao item 2 - obter a *utilidade* daquele item para aquele usuário - podemos simplesmente utilizar o método `predict` do objeto `algoritmo`. Estamos efetivamente *testando* nosso modelo para um caso específico.

In [11]:
algoritmo.predict(uid=1, iid=2)

Prediction(uid=1, iid=2, r_ui=None, est=5, details={'actual_k': 1, 'was_impossible': False})

O resultado aparece em `est`; podemos ver que nosso algoritmo de filtragem colaborativa baseada em usuários e com similaridade de coseno prevê uma utilidade de 5/5 do item 2 para o usuário 1.

Vamos repetir isso para cada usuário - estaremos testando *manualmente* o nosso modelo para cada usuário. Irei re-introduzir a matrix de avaliações, apenas para incluir uma avaliação real ao lado de cada estimativa do nosso modelo.

In [12]:
matrix_avaliacoes = np.array([[5, 3, 0, 4, 4, 0],[1, 0, 0, 3, 0, 0],[0, 0, 0, 1, 0, 0],[4, 0, 0, 5, 0, 2],[0, 0, 5, 4, 0, 1]])

Abaixo, cada linha contém uma estimativa da avaliação/utilidade de um `item` para um `user`, nomeadamente `"est"`. 

`"r_ui"` representa a avaliação *real* que um usuário deu a um item; caso o usuário não tenha avaliado um item, o valor de `"r_ui"` será `0`.

In [13]:
for u in range(5):
    for i in range(6):
        print(algoritmo.predict(u, i, r_ui=matrix_avaliacoes[u, i]))

user: 0          item: 0          r_ui = 5.00   est = 3.46   {'actual_k': 3, 'was_impossible': False}
user: 0          item: 1          r_ui = 3.00   est = 3.00   {'actual_k': 1, 'was_impossible': False}
user: 0          item: 2          r_ui = 0.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 0          item: 3          r_ui = 4.00   est = 3.41   {'actual_k': 5, 'was_impossible': False}
user: 0          item: 4          r_ui = 4.00   est = 4.00   {'actual_k': 1, 'was_impossible': False}
user: 0          item: 5          r_ui = 0.00   est = 1.49   {'actual_k': 2, 'was_impossible': False}
user: 1          item: 0          r_ui = 1.00   est = 3.22   {'actual_k': 3, 'was_impossible': False}
user: 1          item: 1          r_ui = 0.00   est = 3.00   {'actual_k': 1, 'was_impossible': False}
user: 1          item: 2          r_ui = 0.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 1          item: 3          r_ui = 3.00   est = 3.36   {'actual_k': 5, 'was_

#### 7. Avaliando a Performance do Modelo

Antes de gerar recomendações a partir dessas estimativas, é importante avaliarmos a performance de nosso modelo.

Existem diversas maneiras de avaliar a performance de nosso modelo, compondo métricas diferentes com partições diferentes dos dados de avaliações disponíveis.

Uma dessas maneiras é chamada de Validação Cruzada (*Cross Validation*, do Inglês), implementada abaixo.

In [14]:
cross_validate(algoritmo, dados, measures=["RMSE", "MAE"], cv=2, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    2.0354  1.8369  1.9361  0.0993  
MAE (testset)     1.5714  1.6190  1.5952  0.0238  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([2.03540098, 1.83688586]),
 'test_mae': array([1.57142857, 1.61904762]),
 'fit_time': (0.0001456737518310547, 3.147125244140625e-05),
 'test_time': (0.00015616416931152344, 6.389617919921875e-05)}

Acima conseguimos ver duas métricas de avaliação (`RMSE`, *Root Mean Squared Error*; e `MAE`, *Mean Absolute Error*), aplicadas em duas divisões de nossos dados (`cv=2`, `Fold 1` e `Fold 2`), além do tempo de treino e teste (`Test time` e `Fit time`), que são "gastos computacionais".

A medida que nossos modelos se tornam mais complexos e a disponibilidade de dados aumenta, validações cruzadas acabam providenciando uma maneira rápida de avaliar escalabilidade e acurácia: elas testam o quão bem nosso modelo conhece segmentos dos dados existentes de avaliações, e o quão escalável as metodologias atuais são.

#### 8. Gerando recomendações

Uma maneira de utilizar nosso modelo é a partir da construção de um Anti-Conjunto de Dados para Teste. Na parte 6, testamos nosso modelo para cada combinação de usuário e item do sistema de forma manual. Porém, podemos criar um Conjunto de Dados para Teste, que consiste de todas as avaliações conhecidas do sistema, ou um Anti-Conjunto de Dados para Teste, que neste caso consiste de todas as avaliações que o sistema desconhece. 

É evidente que tal Anti-Conjunto terá apenas dados de usuários e itens; afinal, avaliações desconhecidas não possuem nenhum dado. Porém, é exatamente desse tipo de informação que precisamos: dizer ao modelo quais combinações de usuários e itens não possuímos avaliações, e fazer com o modelo estime essas avaliações. Em uma etapa subsequente, podemos usar essas estimativas para gerar recomendações de itens que usuários desconhecem, mas que provavelmente irão se interessar por.

Abaixo, criamos um Anti-Conjunto de Dados para Teste, que consiste de todas as avaliações não disponíveis no Conjunto de Dados para Treinamento. Optamos por utilizar o conjunto de dados para Treinamento pois, novamente, ele consiste de todas as avaliações conhecidas do sistema.

In [15]:
anti_conjunto_dados_teste = conjunto_dados_treinamento.build_anti_testset()

Podemos visualizar esse anti-conjunto:

In [16]:
for avaliacao in anti_conjunto_dados_teste:
    print(f"Usuário: {avaliacao[0]}, Item: {avaliacao[1]}, Avaliacao: {avaliacao[2]}")

Usuário: 0, Item: 5, Avaliacao: 3.230769230769231
Usuário: 0, Item: 2, Avaliacao: 3.230769230769231
Usuário: 1, Item: 1, Avaliacao: 3.230769230769231
Usuário: 1, Item: 4, Avaliacao: 3.230769230769231
Usuário: 1, Item: 5, Avaliacao: 3.230769230769231
Usuário: 1, Item: 2, Avaliacao: 3.230769230769231
Usuário: 2, Item: 0, Avaliacao: 3.230769230769231
Usuário: 2, Item: 1, Avaliacao: 3.230769230769231
Usuário: 2, Item: 4, Avaliacao: 3.230769230769231
Usuário: 2, Item: 5, Avaliacao: 3.230769230769231
Usuário: 2, Item: 2, Avaliacao: 3.230769230769231
Usuário: 3, Item: 1, Avaliacao: 3.230769230769231
Usuário: 3, Item: 4, Avaliacao: 3.230769230769231
Usuário: 3, Item: 2, Avaliacao: 3.230769230769231
Usuário: 4, Item: 0, Avaliacao: 3.230769230769231
Usuário: 4, Item: 1, Avaliacao: 3.230769230769231
Usuário: 4, Item: 4, Avaliacao: 3.230769230769231


Note que o anti conjunto, supostamente, contém todas as avaliações desconhecidas pelo sistema. Mas, de qualquer maneira, o Surprise retornou uma avaliação de `3.230769230769231`. Esse número corresponde a média de todas as avaliações do sistema: ao não conhecer a avaliação de um certo usuário para um certo item, o Surprise simplesmente completou esse valor com a média. Ele não será útil para nós, mas é importante saber de sua existência.

Podemos testar o nosso modelo em nosso anti-conjunto com apenas uma linha:

In [17]:
estimativas = algoritmo.test(anti_conjunto_dados_teste)

E claro, podemos também mostrar todas as estimativas de avaliações desconhecidas.

In [18]:
for i in estimativas: print(i, type(i))

user: 0          item: 5          r_ui = 3.23   est = 1.00   {'actual_k': 1, 'was_impossible': False} <class 'surprise.prediction_algorithms.predictions.Prediction'>
user: 0          item: 2          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'User and/or item is unknown.'} <class 'surprise.prediction_algorithms.predictions.Prediction'>
user: 1          item: 1          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'} <class 'surprise.prediction_algorithms.predictions.Prediction'>
user: 1          item: 4          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'} <class 'surprise.prediction_algorithms.predictions.Prediction'>
user: 1          item: 5          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'} <class 'surprise.prediction_algorithms.predictions.Prediction'>
user: 1          item: 2          r_ui = 3.23   est = 2.57   {'was_impossible': True, '

Dispondo desses dados, podemos realizar uma tarefa ubíqua em sistemas de recomendação: recomendar os N itens mais bem avaliados para um usuário (recomendando apenas os itens que o usuário desconhece).

In [19]:
def recomendar_n_itens_mais_avaliados(estimativas: list, n=2) -> dict:
    """
    Dado uma lista de estimativas (objetos Prediction do Surprise),
    retorna um dicionário onde as chaves são usuários e os valores
    são os ids dos N itens mais bem avaliados para cada usuário.

    Args:
        estimativas: lista de objetos Prediction gerados pelo Surprise.
        n: int do número de itens mais bem avaliados retornados.

    Returns:
        n_itens_mais_avaliados: dicionário especificado acima.
    """

    # Criar o dicionário final, atualmente vazio.
    n_itens_mais_avaliados = defaultdict(list) 

    # Adicionar todos itens estimados para o dicionário.
    for id_usuario, id_item, _, estimativa, _ in estimativas:
        n_itens_mais_avaliados[id_usuario].append((id_item, estimativa))

    # Organizar a lista de cada usuário contido no dicionário, e pegar 
    # apenas os n itens mais bem avaliados.
    for id_usuario in n_itens_mais_avaliados:
        estimativas_usuario = n_itens_mais_avaliados[id_usuario]
        estimativas_usuario.sort(key=lambda x: x[1], reverse=True) 
        n_itens_mais_avaliados[id_usuario] = estimativas_usuario[:n]

    return n_itens_mais_avaliados

In [20]:
top_n = recomendar_n_itens_mais_avaliados(estimativas, n=2)

In [21]:
for u in range(5):
    print(f"Recomendações para o Usuário {u}:")
    for recomendacao_usuario in top_n[u]:
        print(f"Item {recomendacao_usuario[0]}")

Recomendações para o Usuário 0:
Item 2
Item 5
Recomendações para o Usuário 1:
Item 1
Item 4
Recomendações para o Usuário 2:
Item 4
Item 1
Recomendações para o Usuário 3:
Item 1
Item 4
Recomendações para o Usuário 4:
Item 4
Item 1


#### 9. Salvando um Modelo e Estimativas para Utilização Futura

Podemos utilizar o módulo `dump` do Surprise para salvar/carregar nosso modelo e estimativas.

Para salvar nosso modelo e estimativas em um arquivo `modelo_e_estimativas.pkl`, podemos utilizar essa linha:

In [22]:
dump.dump("modelo_e_estimativas.pkl", predictions=estimativas, algo=algoritmo)

Para carregar um modelo salvo em um arquivo `modelo_e_estimativas.pkl`, podemos utilizar essa linha:

In [23]:
est, modelo = dump.load("modelo_e_estimativas.pkl")

In [24]:
for i in est: print(i)
print(modelo)

user: 0          item: 5          r_ui = 3.23   est = 1.00   {'actual_k': 1, 'was_impossible': False}
user: 0          item: 2          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1          item: 1          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 1          item: 4          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 1          item: 5          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 1          item: 2          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 2          item: 0          r_ui = 3.23   est = 2.57   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 2          item: 1          r_ui = 3.23   est = 3.00   {'actual_k': 1, 'was_impossible': False}
user: 2          item: 4          r_ui = 3.23   est = 4.

#### Repetição: Filtragem Colaborativa baseada em Itens

Agora vamos fazer o mesmo, mas usando filtragem colaborativa baseada em itens. Basta alterar a `configuracaoes["user_based"]` para `False`

In [25]:
from surprise import Dataset, Reader, KNNBasic, dump

from collections import defaultdict

import numpy as np
import pandas as pd

avaliacoes = pd.DataFrame({
    "usuario": [0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 4],
    "item": [0, 1, 3, 4, 0, 3, 3, 0, 3, 5, 2, 3, 5],
    "avaliacao": [5, 3, 4, 4, 1, 3, 1, 4, 5, 2, 5, 4, 1],
})

leitor = Reader(rating_scale=(1, 5))

dados = Dataset.load_from_df(avaliacoes, leitor)

conjunto_dados_treinamento = dados.build_full_trainset()

configuracoes = {
    "name": "cosine",
    # Isso significa que estaremos utilizando filtragem colaborativa baseada em itens.
    "user_based": False,
}

algoritmo = KNNBasic(sim_options=configuracoes)

algoritmo.fit(conjunto_dados_treinamento)

anti_conjunto_dados_teste = conjunto_dados_treinamento.build_anti_testset()

estimativas = algoritmo.test(anti_conjunto_dados_teste)

dump.dump("modelo_e_estimativas.pkl", predictions=estimativas, algo=algoritmo)

def recomendar_n_itens_mais_avaliados(estimativas: list, n=2) -> dict:
    """
    Dado uma lista de estimativas (objetos Prediction do Surprise),
    retorna um dicionário onde as chaves são usuários e os valores
    são os ids dos N itens mais bem avaliados para cada usuário.

    Args:
        estimativas: lista de objetos Prediction gerados pelo Surprise.
        n: int do número de itens mais bem avaliados retornados.

    Returns:
        n_itens_mais_avaliados: dicionário especificado acima.
    """

    # Criar o dicionário final, atualmente vazio.
    n_itens_mais_avaliados = defaultdict(list) 

    # Adicionar todos itens estimados para o dicionário.
    for id_usuario, id_item, _, estimativa, _ in estimativas:
        n_itens_mais_avaliados[id_usuario].append((id_item, estimativa))

    # Organizar a lista de cada usuário contido no dicionário, e pegar 
    # apenas os n itens mais bem avaliados.
    for id_usuario in n_itens_mais_avaliados:
        estimativas_usuario = n_itens_mais_avaliados[id_usuario]
        estimativas_usuario.sort(key=lambda x: x[1], reverse=True) 
        n_itens_mais_avaliados[id_usuario] = estimativas_usuario[:n]

    return n_itens_mais_avaliados

top_n = recomendar_n_itens_mais_avaliados(estimativas, n=2)

for u in range(5):
    print(f"Recomendações para o Usuário {u}:")
    for recomendacao_usuario in top_n[u]:
        print(f"Item {recomendacao_usuario[0]}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Recomendações para o Usuário 0:
Item 5
Item 2
Recomendações para o Usuário 1:
Item 2
Item 1
Recomendações para o Usuário 2:
Item 0
Item 1
Recomendações para o Usuário 3:
Item 1
Item 4
Recomendações para o Usuário 4:
Item 1
Item 4
